# Dye2, Greenland 

**Station Name:** gls1

**Location:**  Dye2, Qeqqata Province, Greenland 

**Archive:**  [EarthScope Consortium](http://www.earthscope.org), [SOPAC](http://sopac-csrc.ucsd.edu/index.php/sopac/)

**DOI:**  [https://doi.org/10.7283/T5WS8RDB](https://doi.org/10.7283/T5WS8RDB)

**Ellipsoidal Coordinates:**

- Latitude: 66.47940

- Longitude:  -46.31015

- Height: 2150 m

[Station Page at EarthScope](https://www.unavco.org/instrumentation/networks/status/nota/overview/gls1)

[Station Page at Nevada Geodetic Laboratory](http://geodesy.unr.edu/NGLStationPages/stations/GLS1.sta)

[Google Maps Link](https://goo.gl/maps/391a7h2HpacAa59u8) 

<img src="https://gnss-reflections.org/static/images/GLS1.jpg" width="400">
<BR>


In [ ]:
from gnssrefl.daily_avg_cl import daily_avg
from gnssrefl.gnssir_input import make_gnssir_input
from gnssrefl.gnssir_cl import gnssir
from gnssrefl.quickLook_cl import quicklook
from gnssrefl.rinex2snr_cl import rinex2snr
from gnssrefl.installexe_cl import installexe
import ipywidgets as widgets
import os

from pathlib import Path

from gnssrefl.utils import check_environment, set_environment, get_sys

notebook_dir = Path.cwd().parents[1]

# Making sure environment variables are set - this is required to run the gnssrefl code
exists = check_environment()
if exists == False:
    set_environment(
        refl_code=str(notebook_dir),
        orbits=str(notebook_dir / "orbits"),
        exe=str(notebook_dir / "exe"),
    )

# If you are running this locally - make sure the items in the exe folder have execution permissions

# Set local variable of refl_code location
refl_code_loc = os.environ["REFL_CODE"]
print("refl_code location:", refl_code_loc)

# import gnssrefl functions


# import the crx2rnx file which is dependent on your working OS - this is required to run the gnssrefl code
# If in docker environment, then you do not need to download crxnrnx
try:
    os.environ["DOCKER"]
except KeyError:
    sys = get_sys()
    installexe(sys)

**Fast Mode:** 
this will download data weekly instead of daily

fast mode for this notebook: ~5 minutes

In [ ]:
weekly = widgets.Checkbox(
    value=True, description="Fast Mode", disabled=False, indent=False
)
weekly

Deselect the Fast Mode checkbox to deactivate fast mode.

## Data Summary

Station gls1 was installed at [Dye2](http://greenlandtoday.com/dye-2-a-relic-from-a-not-so-distant-past/?lang=en) on the Greenland Ice Sheet in 2011. 
The antenna is mounted on a long pole; approximately 3.5-meter of the pole was above the ice at the time of installation. 
The receiver at the site only consistently tracks legacy GPS signals. A detailed discussion of the monument and 
data from the station can be found in [Larson, MacFerrin, and Nylen (2020)](https://tc.copernicus.org/articles/14/1985/2020/tc-14-1985-2020.pdf). 
The latest position time series for gls1 can be retrieved 
from the [Nevada Geodetic Laboratory](http://geodesy.unr.edu/gps_timeseries/tenv3/IGS14/GLS1.tenv3). 
We also have a utility you can use: **download_unr**

As gls1 is on an ice sheet and the ice surface is relatively smooth in all directions, it 
is unlikely that a complicated azimuth mask will be required.
gls1 was originally installed with an elevation mask of 7 degrees, which is suboptimal for reflections research.
Even though the mask was later removed, we will use 7 degrees as the minimum elevation angle for all our analysis.
Similarly, even though the site managers later changed to enable L2C tracking, to ensure that 
a consistent dataset is being used, we will only use L1 data. 

In [ ]:
%%html
<iframe src="https://gnss-reflections.org/api?example=gls1" width="1000" height="950"></iframe>

## quickLook 

Our ultimate goal in this use case is to analyze one year of data. We have chosen the year 
2012 because there was a large melt event on the ice sheet. In order to set the proper
quality control parameters, we will use **quickLook** for one day. First we need to translate 
one day of RINEX data using **rinex2snr**. We will use day of year 100:

In [ ]:
station = "gls1"
year = 2012
doy = 100

In [ ]:
# To understand what rinex2snr returns, you can uncomment the next line of code to learn more about this function
# and it's default parameters
# rinex2snr?
rinex2snr(station, year, doy)

In [ ]:
values, metrics = quicklook(station, year, doy=doy, plt=True)

The first plot is a geographically oriented-summary of the frequency content of the GPS data.
The peaks in these periodograms tell us how high the GPS antenna is above the ice surface.
The peaks are associated with a reflector height (RH) of ~2.5 meters. [For more details on quicklook output](https://github.com/kristinemlarson/gnssrefl/blob/master/docs/quickLook_desc.md).

The next plot shows results with respect to azimuth angle.  The top plot is RH and the other 
two are quality control measures: peak amplitude and peak to noise ratio.

In the top plot we see that the retrieved reflector heights are consistent at all azimuths.
Retrievals for azimuths between 340 degrees and 40 degrees are consistently marked as not having
met quality control settings.From the center plot we can see that a peak2noise QC metric of 3 is reasonable. 
Similarly, the amplitudes (bottom plot) are generally larger than 10, so 8 is an acceptable minimum value.

Compare that to its level when the site was installed in the year 2011:

In [ ]:
rinex2snr(station, year=2011, doy=271, archive="unavco", weekly=weekly.value)

values, metrics = quicklook(station, 2011, doy=271, plt=True)

## Measure Snow Accumulation in 2012

The first step is to make SNR files for the year 2012:

This should take ~5 minutes

In [ ]:
rinex2snr(station, year, 1, doy_end=366, weekly=weekly.value)

We will next analyze a year of L1 GPS reflection data from this site. We will use the default minimum and maximum 
reflector height values (0.4 and 6 meters). But for the reasons previously stated, we will set a minimum elevation angle 
of 7 degrees. We also specify that we only want to use the L1 data and set peak2noise and a mimimum
amplitude for the periodograms. We will also exclude a bit of the northern tracks by setting our azimuth range to 40-330 degress.
We use the utility **make_gnssir_input** to set and store these analysis settings:

In [ ]:
make_gnssir_input(station, e1=7, peak2noise=3,
                  ampl=8, l1=True, azlist2=[40, 330])

Note: the removal of these azimuths is more related to the GPS satellite inclination than local conditions at gls1.

Now that you have SNR files and input ready, you can go ahead and estimate reflector heights for the year 2012:

In [ ]:
year = 2012
doy = 1
doy_end = 366
plot = False
gnssir(station, year, doy, doy_end=doy_end, plt=plot)

We will use the **daily_avg** tool to compute a daily average RH. A median filter is set to 0.25 meters 
and 30 individual tracks are required in order to recover a daily average:

In [ ]:
daily_avg(station, medfilter=0.25, ReqTracks=30, txtfile="gls1-dailyavg.txt")

Four plots are returned. The first is all tracks. The second shows the number of tracks used in the daily average. The third is the average reflection amplitude. Finally, the average RH each day for the year 2012.

This data shown in the last plot show you long-term accumulation as well as relatively small snow accumulation events. The overall 
plot is dominated by the large melt event in the summer.

**Things to think about:**

* Why do the number of useable tracks drop drastically at various times in the year?

* Why are the number of tracks retrieved in the summer days consistently higher in number than 
in other times of the year? What is different about the surface in the summer of 2012?

* How would you find out whether this year was anomalously large melt?  

* Try comparing the GNSS-IR results with the [validation data](https://tc.copernicus.org/articles/14/1985/2020/tc-14-1985-2020.pdf)

The original [J. Glaciology paper](https://www.kristinelarson.net/wp-content/uploads/2015/10/LarsonWahrKuipers_2015.pdf ) discussing this site.